In [2]:
import os
import sys
src_dir=os.path.abspath("../src")
sys.path.append(src_dir)

In [17]:
" set filesystem variables"
project_root = os.path.abspath('../notebooks')
sys.path.append(project_root)


In [20]:
# Obtenir le chemin du dossier parent
projet_path = os.path.abspath(os.path.join(project_root, os.pardir))

print(projet_path)

/home/onyxia/work/ProjetNlpavril2024/Project1_odione/classification_by_LLM_DL


In [71]:
# retireve data from s3/minio
# Charger la configuration
import yaml
config_path = projet_path+'/config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

file_path_in_s3 = config['data_for_prediction']
file_name_in_local="data_for_prediction.csv"

In [72]:
file_path_in_s3

'Socface_Data/data_for_prediction.csv'

In [76]:
from data import make_data_good1 as md

In [77]:
df=md.get_data(file_name_in_local,file_path_in_s3,sep=",")

 Pour le dataset :  data_for_prediction.csv 

shape (10, 5) 



In [78]:
df.head(5)

,Unnamed: 0,subject_line,groundtruth,prediction,sex
0,0,ebb26ada-044c-4c62-9dbc-a9c8d505d31c,surname: Chardon firstname: Marie occupation: ...,nom: Chardon prénom: Marie date_naissance: 30 ...,femme
1,1,338496f5-e4ca-43ac-aa5c-429cb3f6ac00,surname: Lhopital firstname: Louis-Jean occupa...,nom: Lhopital prénom: Louis Jean date_naissanc...,homme
2,2,e6a279da-9b6f-4f49-b498-64857bc50d1e,surname: Papin firstname: Marie occupation: id...,nom: Pyrin prénom: Marie date_naissance: 55 re...,femme
3,3,7534deca-39e8-4f00-be17-c12460015de1,surname: Lavocat firstname: Marie link: femme ...,nom: Lavocat prénom: Marie date_naissance: 187...,femme
4,4,ef334a66-a504-418a-9872-e7c9db923488,surname: Benne firstname: Marguerite age: 78,nom: Benne prénom: Marguerite date_naissance: ...,femme


In [ ]:
# feature ingeneering on predicted data

In [98]:
from features import make_featur_good7 as mft

In [89]:
input_df=df.drop(["sex"],axis=1)
filename="data_for_prediction"
input_df.head(3)

,Unnamed: 0,subject_line,groundtruth,prediction
0,0,ebb26ada-044c-4c62-9dbc-a9c8d505d31c,surname: Chardon firstname: Marie occupation: ...,nom: Chardon prénom: Marie date_naissance: 30 ...
1,1,338496f5-e4ca-43ac-aa5c-429cb3f6ac00,surname: Lhopital firstname: Louis-Jean occupa...,nom: Lhopital prénom: Louis Jean date_naissanc...
2,2,e6a279da-9b6f-4f49-b498-64857bc50d1e,surname: Papin firstname: Marie occupation: id...,nom: Pyrin prénom: Marie date_naissance: 55 re...


In [99]:
preproced_df=mft.preprocess_data(input_df,filename)

data_for_prediction  :  (10, 2)
well done


In [100]:
preproced_df.head(3)

prenom
marie         [tensor(101), tensor(2053), tensor(2213), tens...
louis jean    [tensor(101), tensor(2053), tensor(2213), tens...
marie         [tensor(101), tensor(2053), tensor(2213), tens...
Name: input_ids, dtype: object

In [102]:
len(preproced_df[0])

70

In [ ]:
# Retrieve best model

In [49]:
import torch
from models import  CV_train28 as cv# Loading the saved model
# Calculer input_len basé sur les données
#input_len = x_train.shape[1]
input_len=70

# Charger la configuration du modèle
config = torch.load(projet_path+"/models/best_model_config.pth")

# Ajouter input_len à la configuration
config['input_len'] = input_len

# Initialiser le modèle avec les paramètres de la configuration
loaded_model = cv.classe_model_3_LAYERS(**config)

# Charger les poids du modèle
loaded_model.load_state_dict(torch.load(projet_path+"/models/best_model.pth"))

# Mettre le modèle en mode évaluation
loaded_model.eval()

print("Model loaded successfully")

Model loaded successfully


In [50]:
type(loaded_model)

models.CV_train28.classe_model_3_LAYERS

In [51]:
loaded_model

classe_model_3_LAYERS(
  (couche_lineaire_1): Linear(in_features=70, out_features=11, bias=True)
  (couche_lineaire_2): Linear(in_features=11, out_features=10, bias=True)
  (couche_lineaire_3): Linear(in_features=10, out_features=11, bias=True)
  (couche_lineaire_4): Linear(in_features=11, out_features=2, bias=True)
)

In [ ]:
# make prediction

In [46]:
from features import make_fetaure20

In [47]:
(x_train,y_train),(x_test,y_test)=make_fetaure20.preprocess_data()

 Pour le dataset : Firstname 

shape (6946, 3) 

 Pour le dataset : Transcriptions 

shape (241, 4)


 Train : (192,) (192,)
 Test : (49,) (49,)
well done


In [59]:
# Faire des prédictions
with torch.no_grad():  # Désactiver la grad pour les prédictions
    predictions = loaded_model(x_train)
    _, predicted = torch.max(predictions, 1)

In [60]:
type(predicted)

torch.Tensor

In [62]:
predicted.shape

torch.Size([192])

In [63]:
predicted_values=predicted.numpy()

In [64]:
predicted_values

array([1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1])

In [103]:
# Faire des prédictions sur 
with torch.no_grad():  # Désactiver la grad pour les prédictions
    predictions1 = loaded_model(preproced_df)
    _, predicted1 = torch.max(predictions1, 1)

In [104]:
predicted1

tensor([1, 1, 1, 1, 0, 1, 0, 1, 0, 0])

In [108]:
projet_path

'/home/onyxia/work/ProjetNlpavril2024/Project1_odione/classification_by_LLM_DL'